In [ ]:
# ============================================================================
# LAB 2: AZURE VM PERFORMANCE MONITOR
# Topics: Functions, Loops, Control Flow, NumPy
# ============================================================================

# ============================================================================
# CELL 1: Setup and Generate VM Metrics
# ============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

print("=" * 70)
print("🖥️  AZURE VM PERFORMANCE MONITORING LAB")
print("=" * 70)

# Simulate 5 VMs with 24 hours of metrics (readings every hour)
np.random.seed(42)
num_vms = 5
hours = 24

vm_names = [f'vm-prod-{i+1:02d}' for i in range(num_vms)]

# Generate realistic VM metrics
vm_metrics = {}
for vm_name in vm_names:
    # Simulate CPU with daily patterns (higher during business hours)
    base_cpu = np.random.uniform(30, 50)
    time_factor = np.sin(np.linspace(0, 2*np.pi, hours)) * 20
    noise = np.random.normal(0, 5, hours)
    cpu = np.clip(base_cpu + time_factor + noise, 0, 100)
    
    # Memory usage (generally more stable)
    base_memory = np.random.uniform(40, 70)
    memory = np.clip(base_memory + np.random.normal(0, 8, hours), 0, 100)
    
    # Disk I/O (sporadic spikes)
    disk = np.random.exponential(scale=30, size=hours)
    disk = np.clip(disk, 0, 100)
    
    # Network traffic
    network = np.random.gamma(shape=2, scale=15, size=hours)
    network = np.clip(network, 0, 100)
    
    vm_metrics[vm_name] = {
        'cpu': cpu,
        'memory': memory,
        'disk': disk,
        'network': network
    }

print(f"✅ Generated metrics for {num_vms} VMs over {hours} hours")
print(f"📊 Metrics tracked: CPU, Memory, Disk I/O, Network")


# ============================================================================
# CELL 2: Create Monitoring Functions
# ============================================================================
print("\n" + "=" * 70)
print("🔧 CREATING MONITORING FUNCTIONS")
print("=" * 70)

def calculate_statistics(data):
    """Calculate key statistics for a metric array"""
    return {
        'mean': np.mean(data),
        'median': np.median(data),
        'std': np.std(data),
        'min': np.min(data),
        'max': np.max(data),
        'p95': np.percentile(data, 95)  # 95th percentile
    }

def check_threshold(value, warning_threshold, critical_threshold):
    """Check if value exceeds thresholds"""
    if value >= critical_threshold:
        return 'CRITICAL', '🔴'
    elif value >= warning_threshold:
        return 'WARNING', '🟡'
    else:
        return 'OK', '🟢'

def analyze_vm_health(vm_name, metrics):
    """Comprehensive VM health analysis"""
    cpu_stats = calculate_statistics(metrics['cpu'])
    memory_stats = calculate_statistics(metrics['memory'])
    
    # Define thresholds
    cpu_status, cpu_icon = check_threshold(cpu_stats['p95'], 70, 85)
    memory_status, memory_icon = check_threshold(memory_stats['p95'], 75, 90)
    
    # Calculate health score (0-100)
    health_score = 100
    if cpu_stats['p95'] > 70:
        health_score -= (cpu_stats['p95'] - 70) * 1.5
    if memory_stats['p95'] > 75:
        health_score -= (memory_stats['p95'] - 75) * 1.5
    health_score = max(0, min(100, health_score))
    
    return {
        'vm_name': vm_name,
        'cpu_status': cpu_status,
        'cpu_icon': cpu_icon,
        'memory_status': memory_status,
        'memory_icon': memory_icon,
        'cpu_avg': cpu_stats['mean'],
        'memory_avg': memory_stats['mean'],
        'cpu_p95': cpu_stats['p95'],
        'memory_p95': memory_stats['p95'],
        'health_score': health_score
    }

def detect_anomalies(data, threshold=2.5):
    """Detect anomalies using standard deviation method"""
    mean = np.mean(data)
    std = np.std(data)
    anomalies = []
    
    for i, value in enumerate(data):
        z_score = abs((value - mean) / std) if std > 0 else 0
        if z_score > threshold:
            anomalies.append({'hour': i, 'value': value, 'z_score': z_score})
    
    return anomalies

def predict_capacity(current_usage, growth_rate=1.02):
    """Predict when VM will reach capacity"""
    days_to_capacity = 0
    usage = current_usage
    
    while usage < 90 and days_to_capacity < 365:
        usage *= growth_rate
        days_to_capacity += 1
    
    return days_to_capacity if days_to_capacity < 365 else None

print("✅ Functions created:")
print("   • calculate_statistics() - Compute mean, median, std, percentiles")
print("   • check_threshold() - Alert level detection")
print("   • analyze_vm_health() - Comprehensive health check")
print("   • detect_anomalies() - Anomaly detection using z-score")
print("   • predict_capacity() - Capacity planning forecast")


# ============================================================================
# CELL 3: Analyze All VMs
# ============================================================================
print("\n" + "=" * 70)
print("📊 ANALYZING ALL VMs")
print("=" * 70)

# Analyze each VM
vm_health_reports = []

for vm_name, metrics in vm_metrics.items():
    health_report = analyze_vm_health(vm_name, metrics)
    vm_health_reports.append(health_report)
    
    print(f"\n{vm_name}:")
    print(f"  CPU:    {health_report['cpu_icon']} {health_report['cpu_status']:8s} | Avg: {health_report['cpu_avg']:5.1f}% | P95: {health_report['cpu_p95']:5.1f}%")
    print(f"  Memory: {health_report['memory_icon']} {health_report['memory_status']:8s} | Avg: {health_report['memory_avg']:5.1f}% | P95: {health_report['memory_p95']:5.1f}%")
    print(f"  Health Score: {health_report['health_score']:.1f}/100")

# Create summary DataFrame
df_health = pd.DataFrame(vm_health_reports)

print("\n" + "=" * 70)
print("🎯 FLEET SUMMARY")
print("=" * 70)
print(f"Total VMs: {len(vm_health_reports)}")
print(f"Critical VMs: {len([r for r in vm_health_reports if r['cpu_status']=='CRITICAL' or r['memory_status']=='CRITICAL'])}")
print(f"Warning VMs: {len([r for r in vm_health_reports if r['cpu_status']=='WARNING' or r['memory_status']=='WARNING'])}")
print(f"Healthy VMs: {len([r for r in vm_health_reports if r['cpu_status']=='OK' and r['memory_status']=='OK'])}")
print(f"Average Fleet Health: {df_health['health_score'].mean():.1f}/100")


# ============================================================================
# CELL 4: Detect Anomalies Using Loops
# ============================================================================
print("\n" + "=" * 70)
print("🔍 ANOMALY DETECTION")
print("=" * 70)

anomaly_count = 0

for vm_name, metrics in vm_metrics.items():
    cpu_anomalies = detect_anomalies(metrics['cpu'])
    memory_anomalies = detect_anomalies(metrics['memory'])
    
    if cpu_anomalies or memory_anomalies:
        anomaly_count += 1
        print(f"\n⚠️  {vm_name}:")
        
        if cpu_anomalies:
            print(f"   CPU Anomalies detected: {len(cpu_anomalies)}")
            for anomaly in cpu_anomalies[:3]:  # Show first 3
                print(f"      Hour {anomaly['hour']:2d}: {anomaly['value']:.1f}% (z-score: {anomaly['z_score']:.2f})")
        
        if memory_anomalies:
            print(f"   Memory Anomalies detected: {len(memory_anomalies)}")
            for anomaly in memory_anomalies[:3]:
                print(f"      Hour {anomaly['hour']:2d}: {anomaly['value']:.1f}% (z-score: {anomaly['z_score']:.2f})")

if anomaly_count == 0:
    print("✅ No anomalies detected across all VMs")
else:
    print(f"\n📊 Total VMs with anomalies: {anomaly_count}/{num_vms}")


# ============================================================================
# CELL 5: Capacity Planning with NumPy
# ============================================================================
print("\n" + "=" * 70)
print("📈 CAPACITY PLANNING FORECAST")
print("=" * 70)

for vm_name, metrics in vm_metrics.items():
    current_cpu = np.mean(metrics['cpu'])
    current_memory = np.mean(metrics['memory'])
    
    # Predict capacity based on 2% monthly growth
    cpu_days = predict_capacity(current_cpu, growth_rate=1.02)
    memory_days = predict_capacity(current_memory, growth_rate=1.02)
    
    print(f"\n{vm_name}:")
    print(f"  Current CPU: {current_cpu:.1f}%")
    print(f"  Current Memory: {current_memory:.1f}%")
    
    if cpu_days:
        print(f"  🔴 CPU will reach 90% in ~{cpu_days} days")
    else:
        print(f"  ✅ CPU capacity sufficient for >1 year")
    
    if memory_days:
        print(f"  🔴 Memory will reach 90% in ~{memory_days} days")
    else:
        print(f"  ✅ Memory capacity sufficient for >1 year")


# ============================================================================
# CELL 6: Advanced NumPy Operations
# ============================================================================
print("\n" + "=" * 70)
print("🧮 ADVANCED PERFORMANCE ANALYTICS")
print("=" * 70)

# Create a 2D array of all CPU metrics (VMs x Hours)
cpu_matrix = np.array([vm_metrics[vm]['cpu'] for vm in vm_names])
memory_matrix = np.array([vm_metrics[vm]['memory'] for vm in vm_names])

# Calculate correlations
print("\n📊 Cross-VM Correlation Analysis:")
cpu_correlation = np.corrcoef(cpu_matrix)
print(f"Average CPU correlation: {np.mean(cpu_correlation[np.triu_indices_from(cpu_correlation, k=1)]):.3f}")

# Find peak usage hours
hourly_avg_cpu = np.mean(cpu_matrix, axis=0)
peak_hour = np.argmax(hourly_avg_cpu)
print(f"\n⏰ Peak Usage Hour: Hour {peak_hour} (avg {hourly_avg_cpu[peak_hour]:.1f}% CPU)")

# Calculate efficiency score (lower is better - means less wasted resources)
target_utilization = 70
efficiency = np.abs(cpu_matrix - target_utilization).mean()
print(f"\n💡 Fleet Efficiency Score: {efficiency:.1f}")
print(f"   (Distance from optimal 70% utilization)")

# Identify underutilized VMs
underutilized = []
for i, vm_name in enumerate(vm_names):
    avg_cpu = np.mean(cpu_matrix[i])
    avg_memory = np.mean(memory_matrix[i])
    if avg_cpu < 30 and avg_memory < 40:
        underutilized.append((vm_name, avg_cpu, avg_memory))

if underutilized:
    print(f"\n💰 Underutilized VMs (candidates for downsizing):")
    for vm, cpu, mem in underutilized:
        print(f"   {vm}: CPU {cpu:.1f}%, Memory {mem:.1f}%")


# ============================================================================
# CELL 7: Visualize Performance Trends
# ============================================================================
print("\n" + "=" * 70)
print("📊 GENERATING VISUALIZATIONS")
print("=" * 70)

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Azure VM Fleet Performance Dashboard', fontsize=16, fontweight='bold')

# Plot 1: CPU Usage Heatmap
im1 = axes[0, 0].imshow(cpu_matrix, cmap='RdYlGn_r', aspect='auto', vmin=0, vmax=100)
axes[0, 0].set_title('CPU Usage Heatmap (%)', fontweight='bold')
axes[0, 0].set_ylabel('VM')
axes[0, 0].set_xlabel('Hour')
axes[0, 0].set_yticks(range(num_vms))
axes[0, 0].set_yticklabels(vm_names)
plt.colorbar(im1, ax=axes[0, 0], label='CPU %')

# Plot 2: Average Metrics by VM
vm_avg_cpu = np.mean(cpu_matrix, axis=1)
vm_avg_memory = np.mean(memory_matrix, axis=1)
x = np.arange(len(vm_names))
width = 0.35
axes[0, 1].bar(x - width/2, vm_avg_cpu, width, label='CPU', color='#0078D4')
axes[0, 1].bar(x + width/2, vm_avg_memory, width, label='Memory', color='#50E6FF')
axes[0, 1].set_title('Average Resource Usage by VM', fontweight='bold')
axes[0, 1].set_ylabel('Usage (%)')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(vm_names, rotation=45, ha='right')
axes[0, 1].legend()
axes[0, 1].axhline(y=70, color='orange', linestyle='--', alpha=0.5, label='Target')
axes[0, 1].grid(axis='y', alpha=0.3)

# Plot 3: Hourly Fleet Average
hours_range = np.arange(hours)
axes[1, 0].plot(hours_range, hourly_avg_cpu, marker='o', linewidth=2, 
                color='#0078D4', label='CPU')
axes[1, 0].fill_between(hours_range, hourly_avg_cpu, alpha=0.3, color='#0078D4')
axes[1, 0].axhline(y=70, color='orange', linestyle='--', label='Warning (70%)')
axes[1, 0].axhline(y=85, color='red', linestyle='--', label='Critical (85%)')
axes[1, 0].set_title('Fleet Average CPU Over Time', fontweight='bold')
axes[1, 0].set_xlabel('Hour')
axes[1, 0].set_ylabel('CPU %')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Plot 4: Health Score Distribution
health_scores = [r['health_score'] for r in vm_health_reports]
axes[1, 1].bar(vm_names, health_scores, color=['#107C10' if s >= 80 else '#FFB900' if s >= 60 else '#E81123' for s in health_scores])
axes[1, 1].set_title('VM Health Scores', fontweight='bold')
axes[1, 1].set_ylabel('Health Score (0-100)')
axes[1, 1].set_xticklabels(vm_names, rotation=45, ha='right')
axes[1, 1].axhline(y=80, color='green', linestyle='--', alpha=0.5, label='Good')
axes[1, 1].axhline(y=60, color='orange', linestyle='--', alpha=0.5, label='Fair')
axes[1, 1].legend()
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('vm_performance_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Dashboard saved as 'vm_performance_dashboard.png'")


# ============================================================================
# CELL 8: Generate Action Report
# ============================================================================
print("\n" + "=" * 70)
print("📋 ACTIONABLE RECOMMENDATIONS")
print("=" * 70)

recommendations = []

# Check each VM and generate recommendations
for i, vm_name in enumerate(vm_names):
    avg_cpu = np.mean(cpu_matrix[i])
    avg_memory = np.mean(memory_matrix[i])
    p95_cpu = np.percentile(cpu_matrix[i], 95)
    p95_memory = np.percentile(memory_matrix[i], 95)
    
    if p95_cpu > 85 or p95_memory > 90:
        recommendations.append({
            'vm': vm_name,
            'priority': 'HIGH',
            'action': 'Scale Up',
            'reason': f'P95 CPU: {p95_cpu:.1f}%, Memory: {p95_memory:.1f}%'
        })
    elif avg_cpu < 20 and avg_memory < 30:
        recommendations.append({
            'vm': vm_name,
            'priority': 'MEDIUM',
            'action': 'Scale Down',
            'reason': f'Underutilized - CPU: {avg_cpu:.1f}%, Memory: {avg_memory:.1f}%'
        })
    elif p95_cpu > 70 or p95_memory > 75:
        recommendations.append({
            'vm': vm_name,
            'priority': 'MEDIUM',
            'action': 'Monitor',
            'reason': f'Approaching limits - P95 CPU: {p95_cpu:.1f}%, Memory: {p95_memory:.1f}%'
        })

if recommendations:
    df_recommendations = pd.DataFrame(recommendations)
    df_recommendations = df_recommendations.sort_values('priority', ascending=False)
    
    print(f"\n🎯 {len(recommendations)} Recommendations:")
    display(df_recommendations)
    
    # Save to CSV
    df_recommendations.to_csv('vm_recommendations.csv', index=False)
    print("\n✅ Recommendations saved to 'vm_recommendations.csv'")
else:
    print("\n✅ All VMs are operating within optimal parameters")

# Summary statistics
print("\n" + "=" * 70)
print("📊 FINAL SUMMARY")
print("=" * 70)
print(f"VMs Analyzed: {num_vms}")
print(f"Time Period: {hours} hours")
print(f"Data Points: {num_vms * hours * 4:,} (4 metrics per VM-hour)")
print(f"Average Fleet CPU: {np.mean(cpu_matrix):.1f}%")
print(f"Average Fleet Memory: {np.mean(memory_matrix):.1f}%")
print(f"Peak Hour: {peak_hour}:00")
print(f"Health Score: {np.mean(health_scores):.1f}/100")
print(f"Recommendations: {len(recommendations)}")

print("\n🎓 LAB 2 COMPLETE!")
print("Skills Demonstrated:")
print("  ✅ Writing reusable functions")
print("  ✅ Using loops for data processing")
print("  ✅ Conditional logic for decision making")
print("  ✅ NumPy for numerical computations")
print("  ✅ Statistical analysis and forecasting")
print("=" * 70)